In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import sys
import pickle
import argparse
import math
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
from functools import partial
import matplotlib.pyplot as plt
plt.switch_backend('agg')
plt.style.use('ggplot')

from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab_Notebooks')
import models

from argparse import ArgumentParser

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
batch_norm = partial(tf.compat.v1.layers.batch_normalization,
                     momentum=0.99,
                     trainable=True,
                     epsilon=1e-5,
                     scale=True)

In [ ]:
class Prior(object):
    def __init__(self, type):
        self.type = type

    def sample(self, shape):
        if self.type == "uniform":
            return np.random.uniform(-1.0, 1.0, shape)
        else:
            return np.random.normal(0, 1, shape)

def conv_out_size_same(size, stride):
    return int(math.ceil(float(size) / float(stride)))

def make_batches(size, batch_size):
    '''Returns a list of batch indices (tuples of indices).
    '''
    return [(i, min(size, i + batch_size)) for i in range(0, size, batch_size)]

def create_image_grid(x, img_size, tile_shape):
    assert (x.shape[0] == tile_shape[0] * tile_shape[1])
    assert (x[0].shape == img_size)

    img = np.zeros((img_size[0] * tile_shape[0] + tile_shape[0] - 1,
                    img_size[1] * tile_shape[1] + tile_shape[1] - 1,
                    3))

    for t in range(x.shape[0]):
        i, j = t // tile_shape[1], t % tile_shape[1]
        img[i * img_size[0] + i : (i + 1) * img_size[0] + i, j * img_size[1] + j : (j + 1) * img_size[1] + j] = x[t]

    return img

In [ ]:
def linear(input, output_dim, scope='linear', stddev=0.01):
    norm = tf.random_normal_initializer(stddev=stddev)
    const = tf.constant_initializer(0.0)
    with tf.variable_scope(scope):
        w = tf.get_variable('weights', [input.get_shape()[1], output_dim], initializer=norm)
        b = tf.get_variable('biases', [output_dim], initializer=const)
        return tf.matmul(input, w) + b

In [ ]:
def conv2d(input_, output_dim,
           k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
           name="conv2d"):
    with tf.variable_scope(name):
        w = tf.get_variable('weights', [k_h, k_w, input_.get_shape()[-1], output_dim],
                            initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding='SAME')

        biases = tf.get_variable('biases', [output_dim], initializer=tf.constant_initializer(0.0))
        # conv = tf.reshape(tf.nn.bias_add(conv, biases), conv.get_shape())

        return tf.nn.bias_add(conv, biases)

In [ ]:
def deconv2d(input_, output_shape,
             k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
             name="deconv2d", with_w=False):
    with tf.variable_scope(name):
        # filter : [height, width, output_channels, in_channels]
        w = tf.get_variable('weights', [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
                            initializer=tf.random_normal_initializer(stddev=stddev))

        try:
            deconv = tf.nn.conv2d_transpose(input_, w, output_shape=output_shape,
                                            strides=[1, d_h, d_w, 1])

        # Support for versions of TensorFlow before 0.7.0
        except AttributeError:
            deconv = tf.nn.deconv2d(input_, w, output_shape=output_shape,
                                    strides=[1, d_h, d_w, 1])

        biases = tf.get_variable('biases', [output_shape[-1]],
                                 initializer=tf.constant_initializer(0.0))
        deconv = tf.reshape(tf.nn.bias_add(deconv, biases), deconv.get_shape())

        if with_w:
            return deconv, w, biases
        else:
            return deconv

In [ ]:
def lrelu(x, alpha=0.2):
    return tf.maximum(x, alpha * x)

In [ ]:
!pip install Pillow==5.0.0
!pip install scipy==1.0.1

     |████████████████████████████████| 14.2 MB 162 kB/s 
  Created wheel for Pillow: filename=Pillow-5.0.0-cp37-cp37m-linux_x86_64.whl size=1066918 sha256=50f85d86acba6c8e0ce539f86b6a22bf34cdd67f38c0b97977ee0d31784d3110
  Stored in directory: /root/.cache/pip/wheels/0c/e9/3f/e1cf5b199032b95130c1e68222992ce83a78a51bdba0577a3b
Successfully built Pillow
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires pillow>=5.3.0, but you have pillow 5.0.0 which is incompatible.
bokeh 2.3.3 requires pillow>=7.1.0, but you have pillow 5.0.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 15.5 MB 158 kB/s 
  Created wheel for scipy: filename=scipy-1.0.1-cp37-cp37m-linux_x86_64.whl size=39978877 sha256=11944af8a70b6bf4c6f10e86b622fe5a469afd89b0e7dff3279e6f219eeac2b8
  Stored in directory: /root/.cache/pip/wheels/23/78/6c/81857cb7b297db9725704452fd48e7ef25e0fc395642aee63c
Successfully built scipy
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.2 requires scipy>=1.2.0, but you have scipy 1.0.1 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.0.1 which is incompatible.
cvxpy 1.0.31 requires scipy>=1.1.0, but you have scipy 1.0.1 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgau

In [ ]:
#from models import MGAN


class MGAN(object):
    """Mixture Generative Adversarial Nets
    """

    def __init__(self,
                 model_name='MGAN',
                 beta=1.0,
                 num_z=128,
                 num_gens=4,
                 d_batch_size=64,
                 g_batch_size=32,
                 z_prior="uniform",
                 same_input=True,
                 learning_rate=0.0002,
                 img_size=(32, 32, 3),  # (height, width, channels)
                 num_conv_layers=3,
                 num_gen_feature_maps=128,  # number of feature maps of generator
                 num_dis_feature_maps=128,  # number of feature maps of discriminator
                 sample_fp=None,
                 sample_by_gen_fp=None,
                 num_epochs=5,
                 random_seed=6789):
        self.beta = beta
        self.num_z = num_z
        self.num_gens = num_gens
        self.d_batch_size = d_batch_size
        self.g_batch_size = g_batch_size
        self.z_prior = Prior(z_prior)
        self.same_input = same_input
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.img_size = img_size
        self.num_conv_layers = num_conv_layers
        self.num_gen_feature_maps = num_gen_feature_maps
        self.num_dis_feature_maps = num_dis_feature_maps
        self.sample_fp = sample_fp
        self.sample_by_gen_fp = sample_by_gen_fp
        self.random_seed = random_seed

    def _init(self):
        self.epoch = 0

        # TensorFlow's initialization
        self.tf_graph = tf.Graph()
        self.tf_config = tf.ConfigProto()
        self.tf_config.gpu_options.allow_growth = True
        self.tf_config.log_device_placement = False
        self.tf_config.allow_soft_placement = True
        self.tf_session = tf.Session(config=self.tf_config, graph=self.tf_graph)

        np.random.seed(self.random_seed)
        with self.tf_graph.as_default():
            tf.set_random_seed(self.random_seed)

    def _build_model(self):
        arr = np.array([i // self.g_batch_size for i in range(self.g_batch_size * self.num_gens)])
        d_mul_labels = tf.constant(arr, dtype=tf.int32)

        self.x = tf.placeholder(tf.float32, [None,
                                             self.img_size[0], self.img_size[1], self.img_size[2]],
                                name="real_data")
        self.z = tf.placeholder(tf.float32, [self.g_batch_size * self.num_gens, self.num_z], name='noise')

        # create generator G
        self.g = self._create_generator(self.z)

        # create sampler to generate samples
        self.sampler = self._create_generator(self.z, train=False, reuse=True)

        # create discriminator D
        d_bin_x_logits, d_mul_x_logits = self._create_discriminator(self.x)
        d_bin_g_logits, d_mul_g_logits = self._create_discriminator(self.g, reuse=True)

        # define loss functions
        self.d_bin_x_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=d_bin_x_logits, labels=tf.ones_like(d_bin_x_logits)),
            name='d_bin_x_loss')
        self.d_bin_g_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=d_bin_g_logits, labels=tf.zeros_like(d_bin_g_logits)),
            name='d_bin_g_loss')
        self.d_bin_loss = tf.add(self.d_bin_x_loss, self.d_bin_g_loss, name='d_bin_loss')
        self.d_mul_loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=d_mul_g_logits, labels=d_mul_labels),
            name="d_mul_loss")
        self.d_loss = tf.add(self.d_bin_loss, self.d_mul_loss, name="d_loss")

        self.g_bin_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                logits=d_bin_g_logits, labels=tf.ones_like(d_bin_g_logits)),
            name="g_bin_loss")
        self.g_mul_loss = tf.multiply(self.beta, self.d_mul_loss, name='g_mul_loss')
        self.g_loss = tf.add(self.g_bin_loss, self.g_mul_loss, name="g_loss")

        # create optimizers
        self.d_opt = self._create_optimizer(self.d_loss, scope='discriminator',
                                            lr=self.learning_rate)
        self.g_opt = self._create_optimizer(self.g_loss, scope='generator',
                                            lr=self.learning_rate)

    def _create_generator(self, z, train=True, reuse=False, name="generator"):
        out_size = [(conv_out_size_same(self.img_size[0], 2),
                     conv_out_size_same(self.img_size[1], 2),
                     self.num_gen_feature_maps)]
        for i in range(self.num_conv_layers - 1):
            out_size = [(conv_out_size_same(out_size[0][0], 2),
                         conv_out_size_same(out_size[0][1], 2),
                         out_size[0][2] * 2)] + out_size

        with tf.variable_scope(name) as scope:
            if reuse:
                scope.reuse_variables()

            z_split = tf.split(z, self.num_gens, axis=0)
            h0 = []
            for i, var in enumerate(z_split):
                h0.append(tf.nn.relu(batch_norm(linear(var, out_size[0][0] * out_size[0][1] * out_size[0][2],
                                                       scope='g_h0_linear{}'.format(i), stddev=0.02),
                                                training=train #,scope="g_h0_bn{}".format(i)
                                                ),
                                     name="g_h0_relu{}".format(i)))

            h = []
            for var in h0:
                h.append(tf.reshape(var, [self.g_batch_size, out_size[0][0], out_size[0][1], out_size[0][2]]))
            h = tf.concat(h, axis=0, name="g_h0_relu")

            for i in range(1, self.num_conv_layers):
                h = tf.nn.relu(
                    batch_norm(
                        deconv2d(h,
                                 [self.g_batch_size  * self.num_gens, out_size[i][0], out_size[i][1], out_size[i][2]],
                                 stddev=0.02, name="g_h{}_deconv".format(i)),
                        training=train,
                        center=False
                        #,scope="g_h{}_bn".format(i)
                        ),
                    name="g_h{}_relu".format(i))

            g_out = tf.nn.tanh(
                deconv2d(h,
                         [self.g_batch_size * self.num_gens, self.img_size[0], self.img_size[1], self.img_size[2]],
                         stddev=0.02, name="g_out_deconv"),
                name="g_out_tanh")
            return g_out

    def _create_discriminator(self, x, train=True, reuse=False, name="discriminator"):
        with tf.variable_scope(name) as scope:
            if reuse:
                scope.reuse_variables()

            h = x
            for i in range(self.num_conv_layers):
                h = lrelu(batch_norm(conv2d(h, self.num_dis_feature_maps * (2 ** i),
                                            stddev=0.02, name="d_h{}_conv".format(i)),
                                     training=train
                                     #,
                                     #scope="d_bn{}".format(i)
                                     ))

            dim = h.get_shape()[1:].num_elements()
            h = tf.reshape(h, [-1, dim])
            d_bin_logits = linear(h, 1, scope='d_bin_logits')
            d_mul_logits = linear(h, self.num_gens, scope='d_mul_logits')
        return d_bin_logits, d_mul_logits

    def _create_optimizer(self, loss, scope, lr):
        params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope)
        opt = tf.train.AdamOptimizer(lr, beta1=0.5)
        grads = opt.compute_gradients(loss, var_list=params)
        train_op = opt.apply_gradients(grads)
        return train_op

    def fit(self, x):
        if (not hasattr(self, 'epoch')) or self.epoch == 0:
            self._init()
            with self.tf_graph.as_default():
                self._build_model()
                self.tf_session.run(tf.global_variables_initializer())

        num_data = x.shape[0] - x.shape[0] % self.d_batch_size
        batches = make_batches(num_data, self.d_batch_size)
        best_is = 0.0
        while (self.epoch < self.num_epochs):
            for batch_idx, (batch_start, batch_end) in enumerate(batches):
                batch_size = batch_end - batch_start

                x_batch = x[batch_start:batch_end]
                if self.same_input:
                    z_batch = self.z_prior.sample([self.g_batch_size, self.num_z]).astype(np.float32)
                    z_batch = np.vstack([z_batch] * self.num_gens)
                else:
                    z_batch = self.z_prior.sample([self.g_batch_size * self.num_gens, self.num_z]).astype(np.float32)

                # update discriminator D
                d_bin_loss, d_mul_loss, d_loss, _ = self.tf_session.run(
                    [self.d_bin_loss, self.d_mul_loss, self.d_loss, self.d_opt],
                    feed_dict={self.x: x_batch, self.z: z_batch})

                # update generator G
                g_bin_loss, g_mul_loss, g_loss, _ = self.tf_session.run(
                    [self.g_bin_loss, self.g_mul_loss, self.g_loss, self.g_opt],
                    feed_dict={self.z: z_batch})

            self.epoch += 1
            print("Epoch: [%4d/%4d] d_bin_loss: %.5f, d_mul_loss: %.5f, d_loss: %.5f,"
                  " g_bin_loss: %.5f, g_mul_loss: %.5f, g_loss: %.5f" % (self.epoch, self.num_epochs,
                                d_bin_loss, d_mul_loss, d_loss, g_bin_loss, g_mul_loss, g_loss))
            self._samples(self.sample_fp.format(epoch=self.epoch+1))
            self._samples_by_gen(self.sample_by_gen_fp.format(epoch=self.epoch+1))

    def _generate(self, num_samples=100):
        sess = self.tf_session
        batch_size = self.g_batch_size * self.num_gens
        num = ((num_samples - 1) // batch_size + 1) * batch_size
        z = self.z_prior.sample([num, self.num_z]).astype(np.float32)
        x = np.zeros([num, self.img_size[0], self.img_size[1], self.img_size[2]],
                     dtype=np.float32)
        batches = make_batches(num, batch_size)
        for batch_idx, (batch_start, batch_end) in enumerate(batches):
            z_batch = z[batch_start:batch_end]
            x[batch_start:batch_end] = sess.run(self.sampler,
                                                feed_dict={self.z: z_batch})
        idx = np.random.permutation(num)[:num_samples]
        x = (x[idx] + 1.0) / 2.0
        return x

    def _samples(self, filepath, tile_shape=(10, 10)):
        if not os.path.exists(os.path.dirname(filepath)):
            os.makedirs(os.path.dirname(filepath))

        num_samples = tile_shape[0] * tile_shape[1]
        x = self._generate(num_samples)
        imgs = create_image_grid(x, img_size=self.img_size, tile_shape=tile_shape)
        import scipy.misc
        scipy.misc.imsave(filepath, imgs)

    def _samples_by_gen(self, filepath):
        if not os.path.exists(os.path.dirname(filepath)):
            os.makedirs(os.path.dirname(filepath))

        num_samples = self.num_gens * 10
        tile_shape = (self.num_gens, 10)

        sess = self.tf_session
        img_per_gen = num_samples // self.num_gens
        x = np.zeros([num_samples, self.img_size[0], self.img_size[1], self.img_size[2]],
                     dtype=np.float32)
        for i in range(0, img_per_gen, self.g_batch_size):
            z_batch = self.z_prior.sample([self.g_batch_size * self.num_gens, self.num_z]).astype(np.float32)
            samples = sess.run(self.sampler, feed_dict={self.z: z_batch})

            for gen in range(self.num_gens):
                x[gen * img_per_gen + i:gen * img_per_gen + min(i + self.g_batch_size, img_per_gen)] = \
                    samples[
                    gen * self.g_batch_size:gen * self.g_batch_size + min(self.g_batch_size, img_per_gen)]

        x = (x + 1.0) / 2.0
        imgs = create_image_grid(x, img_size=self.img_size, tile_shape=tile_shape)
        import scipy.misc
        scipy.misc.imsave(filepath, imgs)

FLAGS = None


def main(_):
    tmp = pickle.load(open("/content/drive/MyDrive/Colab_Notebooks/cifar10_train.pkl", "rb"))
    x_train = tmp['data'].astype(np.float32).reshape([-1, 32, 32, 3]) / 127.5 - 1.
    model = MGAN(
        num_z=FLAGS.num_z,
        beta=FLAGS.beta,
        num_gens=FLAGS.num_gens,
        d_batch_size=FLAGS.d_batch_size,
        g_batch_size=FLAGS.g_batch_size,
        z_prior=FLAGS.z_prior,
        learning_rate=FLAGS.learning_rate,
        img_size=(32, 32, 3),
        num_conv_layers=FLAGS.num_conv_layers,
        num_gen_feature_maps=FLAGS.num_gen_feature_maps,
        num_dis_feature_maps=FLAGS.num_dis_feature_maps,
        num_epochs=FLAGS.num_epochs,
        sample_fp="samples/samples_{epoch:04d}.png",
        sample_by_gen_fp="samples_by_gen/samples_{epoch:04d}.png",
        random_seed=6789)
    model.fit(x_train)


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--num_z', type=int, default=100,
                        help='Number of latent units.')
    parser.add_argument('--beta', type=float, default=0.01,
                        help='Diversity parameter beta.')
    parser.add_argument('--num_gens', type=int, default=10,
                        help='Number of generators.')
    parser.add_argument('--d_batch_size', type=int, default=64,
                        help='Minibatch size for the discriminator.')
    parser.add_argument('--g_batch_size', type=int, default=12,
                        help='Minibatch size for the generators.')
    parser.add_argument('--z_prior', type=str, default="uniform",
                        help='Prior distribution of the noise (uniform/gaussian).')
    parser.add_argument('--learning_rate', type=float, default=0.0002,
                        help='Learning rate.')
    parser.add_argument('--num_conv_layers', type=int, default=3,
                        help='Number of convolutional layers.')
    parser.add_argument('--num_gen_feature_maps', type=int, default=128,
                        help='Number of feature maps of Generator.')
    parser.add_argument('--num_dis_feature_maps', type=int, default=128,
                        help='Number of feature maps of Discriminator.')
    parser.add_argument('--num_epochs', type=int, default=500,
                        help='Number of epochs.')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/normalization.py:308: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  '`tf.layers.batch_normalization` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
W0809 04:12:26.687870 140590547040128 deprecation.py:336] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/normalization.py:534: _colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
I

Epoch: [   1/ 500] d_bin_loss: 0.56341, d_mul_loss: 1.27234, d_loss: 1.83576, g_bin_loss: 2.48522, g_mul_loss: 0.01148, g_loss: 2.49669


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:241: DeprecationWarning:     `imsave` is deprecated!
    `imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:265: DeprecationWarning:     `imsave` is deprecated!
    `imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imwrite`` instead.


Epoch: [   2/ 500] d_bin_loss: 0.46849, d_mul_loss: 0.57453, d_loss: 1.04302, g_bin_loss: 2.56271, g_mul_loss: 0.00420, g_loss: 2.56691
Epoch: [   3/ 500] d_bin_loss: 0.54170, d_mul_loss: 0.22021, d_loss: 0.76191, g_bin_loss: 2.25133, g_mul_loss: 0.00128, g_loss: 2.25261
Epoch: [   4/ 500] d_bin_loss: 0.56257, d_mul_loss: 0.06653, d_loss: 0.62909, g_bin_loss: 2.15285, g_mul_loss: 0.00037, g_loss: 2.15322
Epoch: [   5/ 500] d_bin_loss: 1.23974, d_mul_loss: 0.10832, d_loss: 1.34806, g_bin_loss: 6.11617, g_mul_loss: 0.00036, g_loss: 6.11653
Epoch: [   6/ 500] d_bin_loss: 0.50384, d_mul_loss: 0.03908, d_loss: 0.54291, g_bin_loss: 1.43074, g_mul_loss: 0.00023, g_loss: 1.43097
Epoch: [   7/ 500] d_bin_loss: 0.70218, d_mul_loss: 0.02532, d_loss: 0.72750, g_bin_loss: 1.27874, g_mul_loss: 0.00011, g_loss: 1.27885
Epoch: [   8/ 500] d_bin_loss: 0.83135, d_mul_loss: 0.01071, d_loss: 0.84206, g_bin_loss: 1.22190, g_mul_loss: 0.00024, g_loss: 1.22214
Epoch: [   9/ 500] d_bin_loss: 0.38235, d_mul_lo